<a href="https://colab.research.google.com/github/cleifton/Data_Science/blob/master/Modelo%20Preditivo%20para%20A%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials 
import seaborn as sb 
  
auth.authenticate_user() 
gauth = GoogleAuth() 
gauth.credentials = GoogleCredentials.get_application_default() 
drive = GoogleDrive(gauth)


import plotly.express as px
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from pmdarima import auto_arima #Predictions
import plotly.graph_objs as go
import pandas as pd 
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings('ignore')



In [86]:

link='https://drive.google.com/file/d/1ReCDPXrMMOZ3H7AGgtLXVLC9a4y_AWQz/view?usp=sharing'
id = link.split("/")[-2] 
downloaded = drive.CreateFile({'id':id})  
downloaded.GetContentFile('HistoricalData_1656553572236.csv')     
df = pd.read_csv('HistoricalData_1656553572236.csv') 
df.columns.values[1] = "Close"
df
print(df)

            Date     Close     Volume      Open      High       Low
0     06/28/2022   $137.44   67315330   $142.13  $143.422  $137.325
1     06/27/2022   $141.66   70207910  $142.695   $143.49  $140.965
2     06/24/2022   $141.66   89116840    $139.9   $141.91   $139.77
3     06/23/2022   $138.27   72433770   $136.82   $138.59   $135.63
4     06/22/2022   $135.35   73409230   $134.79   $137.76   $133.91
...          ...       ...        ...       ...       ...       ...
1253  07/06/2017  $35.6825   96441320   $35.755   $35.875  $35.6025
1254  07/05/2017  $36.0225   86201080  $35.9225  $36.1975  $35.6809
1255  07/03/2017   $35.875   57111400    $36.22   $36.325   $35.775
1256  06/30/2017   $36.005   91947840  $36.1125    $36.24   $35.945
1257  06/29/2017    $35.92  125802080  $36.1775  $36.2825    $35.57

[1258 rows x 6 columns]


In [61]:
df.set_index('Date',drop=False, inplace=True)
df.tail()

,Date,Close,Volume,Open,High,Low
Date,,,,,,
07/06/2017,07/06/2017,$35.6825,96441320,$35.755,$35.875,$35.6025
07/05/2017,07/05/2017,$36.0225,86201080,$35.9225,$36.1975,$35.6809
07/03/2017,07/03/2017,$35.875,57111400,$36.22,$36.325,$35.775
06/30/2017,06/30/2017,$36.005,91947840,$36.1125,$36.24,$35.945
06/29/2017,06/29/2017,$35.92,125802080,$36.1775,$36.2825,$35.57


In [62]:
df.describe()

,Volume
count,1.258000e+03
mean,1.194045e+08
std,5.483372e+07
min,4.099995e+07
25%,8.344617e+07
50%,1.048975e+08
75%,1.389689e+08
max,4.268848e+08


In [63]:
fig_open = px.line(data_frame=df, x='Date', y=['Open'])
fig_open.update_yaxes( # the y-axis is in dollars
    tickprefix="$", showgrid=False, title='Preço Abertura')
fig_open.update_xaxes(
    title = 'Date')
fig_open.show()

In [87]:

fig_close = px.line(data_frame=df, x='Date', y=['Close'])
fig_close.update_yaxes( # the y-axis is in dollars
    tickprefix="$", showgrid=False, title='Preço Fechamentoe')
fig_close.update_xaxes(
    title = 'Date')
fig_close.show()

In [66]:
df_lastyears = df.loc['06/30/2020':]
df_lastyears.tail()

,Date,Close,Volume,Open,High,Low
Date,,,,,,
07/06/2017,07/06/2017,$35.6825,96441320,$35.755,$35.875,$35.6025
07/05/2017,07/05/2017,$36.0225,86201080,$35.9225,$36.1975,$35.6809
07/03/2017,07/03/2017,$35.875,57111400,$36.22,$36.325,$35.775
06/30/2017,06/30/2017,$36.005,91947840,$36.1125,$36.24,$35.945
06/29/2017,06/29/2017,$35.92,125802080,$36.1775,$36.2825,$35.57


In [68]:
fig_close_2 = px.line(data_frame=df_lastyears, x='Date', y=['Close'])
fig_close_2.update_yaxes(tickprefix="$", showgrid=False, title='Preço Fechamento')
fig_close_2.update_xaxes(title = 'Date')
fig_close_2.show()

In [69]:


fig_open_2 = px.line(data_frame=df, x='Date', y=['Open'])

fig_open_2.update_yaxes(tickprefix="$", showgrid=False, title='Open Price')
fig_open_2.update_xaxes(title = 'Date')

fig_open_2.show()



In [71]:


Fig = go.Figure([
                 go.Scatter(
                     name = 'Close',
                     x = df_lastyears.index,
                     y = df_lastyears['Close'],
                     mode= 'markers'
                 ),
                 go.Scatter(
                     name = 'Open',
                     x = df_lastyears.index,
                     y = df_lastyears['Open'],
                     mode= 'markers'
                 )

])

Fig.update_layout(
    yaxis_title='Preço',
    yaxis_tickprefix='$',
    title = 'Preço Abertura Fechamento'
)
Fig.update_traces(
    marker = dict(size=2.5)
)

Fig.show()



In [93]:
#Price difference between two consecutive days
df_lastyears['Price_Difference'] = df_lastyears['Close'].shift(-1) - df_lastyears['Close']
df_lastyears = df_lastyears.dropna()

#Let's calculate the daily return by using the expression we wrote before
df_lastyears['Return'] = df_lastyears['Price_Difference'] / df_lastyears['Close']

fig_return = px.line(data_frame=df_lastyears, x=df_lastyears.index, y='Return', title='Daily Return')
fig_return.update_xaxes(title='Date')
fig_return.show()

TypeError: ignored

In [91]:
df_lastyears['MA10_Close'] = df_lastyears['Close'].rolling(10).mean()
df_lastyears['MA10_Open'] = df_lastyears['Open'].rolling(10).mean()

df_lastyears['MA50_Close'] = df_lastyears['Close'].rolling(50).mean()
df_lastyears['MA50_Open'] = df_lastyears['Open'].rolling(50).mean()



DataError: ignored

In [92]:
MA_Fig = go.Figure([
                    
                    go.Scatter(
                        name='MA10 Close',
                        x = df_lastyears.index,
                        y = df_lastyears['MA10_Close']

                    ),
                    go.Scatter(
                        name='MA50 Close',
                        x = df_lastyears.index,
                        y = df_lastyears['MA50_Close']

                    ),
                    go.Scatter(
                        name='Close Price',
                        x = df_lastyears.index,
                        y = df_lastyears['Close'],
                        mode = 'markers'

                    )
])
MA_Fig.update_layout(
    yaxis_title = 'Price',
    title = 'Close Price & MA10, MA50'
)

MA_Fig.update_traces(
    marker = dict(size=4)
)

MA_Fig.show()



KeyError: ignored

In [78]:
train_data, test_data = df_lastyears[0:int(len(df_lastyears)*0.7)], df_lastyears[int(len(df_lastyears)*0.7):]

In [79]:
train_data

,Date,Close,Volume,Open,High,Low
Date,,,,,,
06/30/2020,06/30/2020,$91.2,140223280,$90.02,$91.495,$90
06/29/2020,06/29/2020,$90.445,130646080,$88.3125,$90.5434,$87.82
06/26/2020,06/26/2020,$88.4075,205256840,$91.1025,$91.33,$88.255
06/25/2020,06/25/2020,$91.21,137522520,$90.175,$91.25,$89.3925
06/24/2020,06/24/2020,$90.015,192623400,$91.25,$92.1975,$89.63
...,...,...,...,...,...,...
05/31/2018,05/31/2018,$46.7175,109874800,$46.805,$47.0575,$46.535
05/30/2018,05/30/2018,$46.875,74013640,$46.93,$47,$46.695
05/29/2018,05/29/2018,$46.975,89636440,$46.9,$47.1875,$46.7175


In [80]:
Fig_tt = go.Figure([
                    
                    go.Scatter(
                        name= 'Train closes',
                        x = train_data.index,
                        y = train_data['Close'],
                        mode = 'markers'
                    ),
                    
                    go.Scatter(
                        name = 'Test closes',
                        x = test_data.index, 
                        y = test_data['Close'],
                        mode = 'markers'

                    )
])

Fig.update_layout(
    yaxis_title = 'Price ($)',
    title = 'Training & Test Close Prices'
)
Fig_tt.show()

In [81]:
ex_variables = ['MA50_Close', 'MA50_Open', 'MA10_Open', 'MA10_Close']
train_data = train_data.dropna()
model = auto_arima(
    train_data['Close'],
    exogenous=train_data[ex_variables],
    trace=True, error_action="ignore",
    suppress_warnings=True)

KeyError: ignored

In [82]:
model.fit(train_data['Close'], exogenous=train_data[ex_variables])

NameError: ignored

In [83]:
test_data = test_data.dropna()

In [84]:
predictions = model.predict(n_periods=len(test_data), exogenous=test_data[ex_variables])
test_data['Predictions'] = predictions

NameError: ignored

In [ ]:
pred_Fig  = go.Figure([
                       
                       go.Scatter(
                        name= 'Train closes',
                        x = train_data.index,
                        y = train_data['Close'],
                        mode = 'markers'
                    ),
                    
                    go.Scatter(
                        name = 'Test closes',
                        x = test_data.index, 
                        y = test_data['Close'],
                        mode = 'markers'

                    ),

                    go.Scatter(
                        name = 'Predictions',
                        x = test_data.index,
                        y = test_data['Predictions'],
                        mode = 'lines'
                    )
])

pred_Fig.show()